In [45]:
import cv2
import numpy as np

In [191]:
img = cv2.imread("./screenshots/fsScreen_2022_10_21_14_00_24.png")
img = cv2.resize(img, (1280, 720))
cv2.imshow('image',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [208]:
def roi(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask,vertices,255)
    masked = cv2.bitwise_and(img, mask)
    return masked

img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
kernel_size = 5
blur_gray = cv2.GaussianBlur(img_gray,(kernel_size, kernel_size),0)
cv2.imshow('image',blur_gray)
cv2.waitKey()
cv2.destroyAllWindows()
low_threshold = 80
high_threshold = 265
edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
#roi_vertices = vertices = np.array([[0,500],[10,300], [300,200], [500,200], [800,300], [800,500]], np.int32)
roi_vertices = np.array([[340,700],[340,300], [250,80], [530,80], [880,300], [880,700]], np.int32)
processed_img = roi(edges, [roi_vertices])

cv2.imshow('image',processed_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 60  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 60  # minimum number of pixels making up a line
max_line_gap = 55  # maximum gap in pixels between connectable line segments
line_image = np.copy(img) * 0  # creating a blank to draw lines on

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
lines = cv2.HoughLinesP(processed_img, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)

for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(line_image,(x1,y1),(x2,y2),(255,255,0),5)
        
lines_edges = cv2.addWeighted(img, 0.8, line_image, 1, 0)
cv2.imshow('image',lines_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()